In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils.np_utils import to_categorical # label数组变为one-hot编码
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, CuDNNLSTM
from keras.optimizers import Adam # Adam 优化，加速收敛
from keras.callbacks import ReduceLROnPlateau # tensorboard记录，learning rate decay，模型保存

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import train_test_split # 用于划分训练集和验证集

In [4]:
from label_data import get_labeled_data

In [5]:
seed = 2

In [10]:
def get_train_val_test(dat_name, lab_name):
    train_x, train_y = get_labeled_data(dat_name, lab_name, 'train_')
    test_x, test_y = get_labeled_data(dat_name, lab_name, 'test_')
    train_y = to_categorical(train_y, num_classes = 3)
    test_y = to_categorical(test_y, num_classes = 3)
    val_x, val_y, test_x, test_y = train_test_split(test_x, 
                                                    test_y, 
                                                    test_size = 0.2, 
                                                    random_state = seed)
    return train_x, train_y, val_x, val_y, test_x, test_y

In [11]:
train_x, train_y, val_x, val_y, test_x, test_y = get_train_val_test('A1', 'A3')

In [13]:
ylist = train_y.tolist()
print(ylist[:10])

[[0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0], [0.0, 1.0, 0.0]]


In [ ]:
li = [i for i in range(len(ylist)) if y[i] == 2]

In [ ]:
print(x[li[9]])